# Get-RscMssqlDatabase
Most operations in RSC happen at the database. When you want to perform backup, restore, export, live mount operations, it is all done with the database. Getting the database ID will be important to perform those actions. To be able to get the correct database ID, you will need its location or parent. The locaiton or parent will be either the SQL Server Instance or the Availability Group. 

# Workflow
To get information about the SQL Server Database, you need to do the following
1. Use **Connect-RSC** to Connect to RSC
1. Use **Get-RscCluster** to get the cluster id that the SQL Server is registered. This is akin to the datacenter that the SQL Server is located in. While you can run the next command without the cluster ID, you may get more records back than expected. This is because your SQL Server Host could exist on 2 different cluster. Once as a replicated source and once as the replicated target. 
1. Use **Get-RscMssqlInstance** to get the instance id of the SQL Server based on the name of the instance and the cluster where it is located. 

    *OR*

1. Use **Get-RscMssqlAvailabilityGroup** to get the id of the Availability Group. 
1. Use **Get-RscMssqlDatabase** to get the database ID based on the database name and the SQL Server Instance. 
1. Use **Disconnect-RSC** to disconnect from RSC

## Connect to RSC

In [3]:
Import-Module RubrikSecurityCloud
Connect-RSC

Importing RubrikSecurityCloud module from /Users/Chris.Lumnah/Documents/rubrik-powershell-sdk/Output/RubrikSecurityCloud.psd1.
RSC Toolkit loaded.
Imported 180 cmdlets and functions from RubrikSecurityCloud.
Connect-Rsc: 
Line |
   4 |  Connect-RSC
     |  ~~~~~~~~~~~
     | Could not find: /Users/Chris.Lumnah/.config/powershell


Error: Command failed: SubmitCode: # Import-Module RubrikSecurityCloud ...

## Get the Cluster ID

In [ ]:
$ClusterName = "vault-r-london"
$RscCluster = Get-RscCluster -Name $ClusterName

## Get SQL Server Instance

### Get the Instance Information

In [ ]:
$HostName = "rp-sql19s-001.perf.rubrik.com"
$RscMssqlInstance = Get-RscMssqlInstance -RscCluster $RscCluster -HostName $HostName

### Get the Database Information based on Instance

In [ ]:
$DatabaseName = "AdventureWorks2019"
$RscMssqlDatabase = Get-RscMssqlDatabase -Name $DatabaseName -RscMssqlInstance $RscMssqlInstance
$RscMssqlDatabase

## Get the Availability Group 

### Get the Availability Group Information

In [ ]:
$AvailabilityGroupName = "DEMO_AG1"
$RscMssqlAvailabilityGroup = Get-RscMssqlAvailabilityGroup -AvailabilityGroupName $AvailabilityGroupName -RscCluster $RscCluster

### Get the Database Information based on Availability Group

In [ ]:
$DatabaseName = "AdventureWorks2019"
$RscMssqlDatabase = Get-RscMssqlDatabase -Name $DatabaseName -RscMssqlAvailabilityGroup $RscMssqlAvailabilityGroup
$RscMssqlDatabase

---

# Disconnect from RSC
It is important to have each script you create end by disconnecting from RSC. If you do not, then you will run into issues with token authentication exhaustion. RSC allows a Service Account to have 10 concurrent open connections to RSC. Additional requests will be denied until existing connections are closed. 

You can easily disconnect your connection by running the below statement. 

In [ ]:
Disconnect-Rsc